In [1]:
import os
import time
import pickle

import implicit
import numpy as np
import pandas as pd

from tqdm import tqdm
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors

/Users/grshchnkv/opt/anaconda3/envs/Lastfm-artist-recommender/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Общее

## Метрики

In [2]:
def precision(actual: list, predicted: list) -> float:
    return len(list(set(actual) & set(predicted))) / len(predicted)

In [3]:
def mean_precision(scores: list, n: int) -> float:
    return np.array(scores).sum() / n

## Загрузка данных

In [4]:
artists = pd.read_csv('../data/lastfm_artist_list.csv', index_col='artist_id')['artist_name'].to_dict()

In [5]:
scrobbles = pd.read_csv('../data/lastfm_user_scrobbles.csv')

## Подготовка данных

### Данные для моделей

In [6]:
users_indexes, users_positions = np.unique(scrobbles.values[:,0], return_inverse=True)
artists_indexes, artists_positions = np.unique(scrobbles.values[:,1], return_inverse=True)

In [7]:
scrobbles_sparse = csr_matrix((scrobbles.values[:,2], (users_positions, artists_positions)))
scrobbles_sparse_normalized = normalize(scrobbles_sparse, norm='l2', axis=1)

### Данные для тестирования

In [8]:
validation = scrobbles.groupby('user_id')['artist_id'].apply(list).to_dict()

# Рекомендации

Число похожих для поиска

In [9]:
k = 5

## Baseline. Рекомендация самых популярных

Найдем максимально необходимый топ исполнителей по прослушиваниям

In [10]:
max_users_listens = scrobbles.groupby('user_id').count()['artist_id'].max()
most_listend_artists = scrobbles.groupby('artist_id').sum()['scrobbles'].sort_values()[-k*max_users_listens:].index

In [11]:
pickle.dump(most_listend_artists, open("most_listend_artists.pkl", "wb"))
size = os.path.getsize("most_listend_artists.pkl")

Построим рекомендации

In [12]:
start = time.time()
scores = []
users = 0
for user in tqdm(validation):
    if len(validation[user]) <= 1:
        continue
    scores.append(precision(validation[user], most_listend_artists[-k*len(validation[user]):]))
    users += 1
score = mean_precision(scores, users)
end = time.time() - start

100%|██████████| 1892/1892 [00:00<00:00, 47204.10it/s]


Оценка

In [13]:
print(f"Метрика: {score}")
print(f"Затраченная память: {size} байт")
print(f"Затраченное время: {end} секунд")

Метрика: 0.07037536297107326
Затраченная память: 2271 байт
Затраченное время: 0.061167240142822266 секунд


## Матрица схожести

Построим модель

In [14]:
artists_similarity = scrobbles_sparse_normalized.T * scrobbles_sparse_normalized

In [15]:
pickle.dump(artists_similarity, open("artists_similarity.pkl", "wb"))
size = os.path.getsize("artists_similarity.pkl")

Построим рекомендации

In [16]:
def get_k_similar_matrix(target: int, k: int) -> list:
    return [x+1 for x in artists_similarity[target-1].toarray()[0].argsort()[-k-1:]][:-1]

In [17]:
start = time.time()
scores = []
users = 0
for user in tqdm(validation):
    if len(validation[user]) <= 1:
        continue
    predictions = []
    for artist in validation[user]:
        predictions += get_k_similar_matrix(artist, k)
    scores.append(precision(validation[user], predictions))
    users += 1
score = mean_precision(scores, users)
end = time.time() - start

100%|██████████| 1892/1892 [05:09<00:00,  6.11it/s]


Оценка

In [18]:
print(f"Метрика: {score}")
print(f"Затраченная память: {size} байт")
print(f"Затраченное время: {end} секунд")

Метрика: 0.11348452374549088
Затраченная память: 31825587 байт
Затраченное время: 309.63960695266724 секунд


## kNN

Построим модель

In [19]:
knn = NearestNeighbors(metric = 'cosine', n_neighbors = 200)
knn.fit(scrobbles_sparse_normalized.T)

NearestNeighbors(metric='cosine', n_neighbors=200)

In [20]:
pickle.dump(knn, open("knn.pkl", "wb"))
size = os.path.getsize("knn.pkl")

Построим рекомендации

In [21]:
def get_k_similar_knn(target: int, k: int) -> list:
    distances, indices = knn.kneighbors(scrobbles_sparse_normalized.T[target-1].toarray(), n_neighbors=k+1)
    return [x+1 for x in indices[0]][1:]

In [22]:
start = time.time()
scores = []
users = 0
for user in tqdm(validation):
    if len(validation[user]) <= 1:
        continue
    predictions = []
    for artist in validation[user]:
        predictions += get_k_similar_knn(artist, k)
    scores.append(precision(validation[user], predictions))
    users += 1
score = mean_precision(scores, users)
end = time.time() - start

100%|██████████| 1892/1892 [02:42<00:00, 11.61it/s]


Оценка

In [23]:
print(f"Метрика: {score}")
print(f"Затраченная память: {size} байт")
print(f"Затраченное время: {end} секунд")

Метрика: 0.07468196033949652
Затраченная память: 1183374 байт
Затраченное время: 162.9682400226593 секунд


## implicit CosineRecommender

Построим модель

In [24]:
cos_rec = implicit.nearest_neighbours.CosineRecommender(K=200)
cos_rec.fit(scrobbles_sparse_normalized)

/Users/grshchnkv/opt/anaconda3/envs/Lastfm-artist-recommender/lib/python3.10/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0007498264312744141 seconds
  warnings.warn(
100%|██████████| 17493/17493 [00:00<00:00, 157024.51it/s]


In [25]:
pickle.dump(cos_rec, open("cos_rec.pkl", "wb"))
size = os.path.getsize("cos_rec.pkl")

Построим рекомендации

In [26]:
def get_k_similar_cos(target: int, k: int) -> list:
    indices, distances = cos_rec.similar_items(target-1, N=5, filter_items=[target-1])
    return [x+1 for x in indices]

In [27]:
start = time.time()
scores = []
users = 0
for user in tqdm(validation):
    if len(validation[user]) <= 1:
        continue
    predictions = []
    for artist in validation[user]:
        predictions += get_k_similar_cos(artist, k)
    scores.append(precision(validation[user], predictions))
    users += 1
score = mean_precision(scores, users)
end = time.time() - start

100%|██████████| 1892/1892 [00:13<00:00, 140.26it/s]


Оценка

In [28]:
print(f"Метрика: {score}")
print(f"Затраченная память: {size} байт")
print(f"Затраченное время: {end} секунд")

Метрика: 0.07592253366226333
Затраченная память: 18903830 байт
Затраченное время: 13.493268966674805 секунд


## implicit ALS

Построим модель

In [29]:
als = implicit.als.AlternatingLeastSquares(factors=64)
als.fit(scrobbles_sparse_normalized)

100%|██████████| 15/15 [00:35<00:00,  2.34s/it]


In [30]:
pickle.dump(als, open("als.pkl", "wb"))
size = os.path.getsize("als.pkl")

Построим рекомендации

In [31]:
def get_k_similar_als(target: int, k: int) -> list:
    indices, distances = als.similar_items(target-1, N=5, filter_items=[target-1])
    return [x+1 for x in indices]

In [32]:
start = time.time()
scores = []
users = 0
for user in tqdm(validation):
    if len(validation[user]) <= 1:
        continue
    predictions = []
    for artist in validation[user]:
        predictions += get_k_similar_als(artist, k)
    scores.append(precision(validation[user], predictions))
    users += 1
score = mean_precision(scores, users)
end = time.time() - start

100%|██████████| 1892/1892 [00:32<00:00, 58.71it/s]


Оценка

In [33]:
print(f"Метрика: {score}")
print(f"Затраченная память: {size} байт")
print(f"Затраченное время: {end} секунд")

Метрика: 0.08763938391136546
Затраченная память: 4963084 байт
Затраченное время: 32.22958588600159 секунд
